# Listagem dos arquivos CSV que foram importados para o HDFS

In [1]:
!hdfs dfs -ls /user/hive/warehouse/hist_painel_covidbr/

Found 4 items
-rw-r--r--   3 root supergroup   62492959 2022-08-07 19:23 /user/hive/warehouse/hist_painel_covidbr/HIST_PAINEL_COVIDBR_2020_Parte1_06jul2021.csv
-rw-r--r--   3 root supergroup   76520681 2022-08-07 19:23 /user/hive/warehouse/hist_painel_covidbr/HIST_PAINEL_COVIDBR_2020_Parte2_06jul2021.csv
-rw-r--r--   3 root supergroup   91120916 2022-08-07 19:23 /user/hive/warehouse/hist_painel_covidbr/HIST_PAINEL_COVIDBR_2021_Parte1_06jul2021.csv
-rw-r--r--   3 root supergroup    3046774 2022-08-07 19:23 /user/hive/warehouse/hist_painel_covidbr/HIST_PAINEL_COVIDBR_2021_Parte2_06jul2021.csv


# Criação da tabela particionada por município

In [2]:
from pyspark.sql.types import StructType

schema = (
    StructType()
    .add('regiao', 'string')
    .add('estado', 'string')
    .add('municipio', 'string')
    .add('coduf', 'integer')
    .add('codmun', 'integer')
    .add('codRegiaoSaude', 'integer')
    .add('nomeRegiaoSaude', 'string')
    .add('data', 'timestamp')
    .add('semanaEpi', 'integer')
    .add('populacaoTCU2019', 'integer')
    .add('casosAcumulado', 'integer')
    .add('casosNovos', 'integer')
    .add('obitosAcumulado', 'integer')
    .add('obitosNovos', 'integer')
    .add('Recuperadosnovos', 'integer')
    .add('emAcompanhamentoNovos', 'integer')
    .add('interior/metropolitana', 'integer')
)

df_csv = (
    spark.read
    .schema(schema)
    .option('header', True)
    .option('encoding', 'UTF-8')
    .option('delimiter', ';')
    .format('csv')
    .load('/user/hive/warehouse/hist_painel_covidbr')
)

In [3]:
df_csv.printSchema()

root
 |-- regiao: string (nullable = true)
 |-- estado: string (nullable = true)
 |-- municipio: string (nullable = true)
 |-- coduf: integer (nullable = true)
 |-- codmun: integer (nullable = true)
 |-- codRegiaoSaude: integer (nullable = true)
 |-- nomeRegiaoSaude: string (nullable = true)
 |-- data: timestamp (nullable = true)
 |-- semanaEpi: integer (nullable = true)
 |-- populacaoTCU2019: integer (nullable = true)
 |-- casosAcumulado: integer (nullable = true)
 |-- casosNovos: integer (nullable = true)
 |-- obitosAcumulado: integer (nullable = true)
 |-- obitosNovos: integer (nullable = true)
 |-- Recuperadosnovos: integer (nullable = true)
 |-- emAcompanhamentoNovos: integer (nullable = true)
 |-- interior/metropolitana: integer (nullable = true)



In [ ]:
(df_csv.write
 .mode("overwrite")
 .partitionBy('municipio')
 .saveAsTable('painel_covidbr.hist_por_municipio'))

In [4]:
!hdfs dfs -ls /user/hive/warehouse/painel_covidbr.db

Found 3 items
drwxr-xr-x   - root supergroup          0 2022-08-08 17:16 /user/hive/warehouse/painel_covidbr.db/hist_por_municipio
drwxr-xr-x   - root supergroup          0 2022-08-08 02:48 /user/hive/warehouse/painel_covidbr.db/painel_casos_recuperados_acompanhamento
drwxr-xr-x   - root supergroup          0 2022-08-08 04:44 /user/hive/warehouse/painel_covidbr.db/sintese_casos_estados


# Leitura da tabela particionada

In [5]:
from pyspark.sql.functions import col

In [ ]:
df = (
    spark.read
    .table('painel_covidbr.hist_por_municipio')
    .select(
        col('regiao'),
        col('estado'),
        col('municipio'),
        col('populacaoTCU2019').alias('populacao'),
        col('casosAcumulado'),
        col('casosNovos'),
        col('obitosAcumulado'),
        col('obitosNovos'),
        col('Recuperadosnovos').alias('recuperadosNovos'),
        col('emAcompanhamentoNovos'),
        col('data')
    )
)

# Criação da tabela intermediária de síntese de casos por Estados

Tornou-se necessária por limitações de hardware do equipamento do autor

In [6]:
from pyspark.sql.window import Window
from pyspark.sql.functions import col, desc, lit, row_number

In [ ]:
w_regioes_estados = Window.partitionBy('regiao', 'estado').orderBy(desc('data'))

df_estados = (
    df
    .withColumn('row', row_number().over(w_regioes_estados))
    .filter(col('row') == 1)
    .drop('row')
)

In [ ]:
df_sintese_casos_estados = (
    df_estados
    .withColumn('incidencia', (col('casosAcumulado') / col('populacao')) * lit(100_000))
    .withColumn('mortalidade', (col('obitosAcumulado') / col('populacao')) * lit(100_000))
    .withColumn('letalidade', col('obitosAcumulado') / col('casosAcumulado'))
    .select(
        col('regiao'),
        col('estado'),
        col('recuperadosNovos'),
        col('emAcompanhamentoNovos'),
        col('casosAcumulado'),
        col('casosNovos'),
        col('incidencia'),
        col('obitosAcumulado'),
        col('obitosNovos'),
        col('letalidade'),
        col('mortalidade'),
        col('populacao'),
        col('data')
    )
)

In [ ]:
(df_sintese_casos_estados.write
 .mode('overwrite')
 .partitionBy('estado')
 .saveAsTable('painel_covidbr.sintese_casos_estados'))

In [15]:
!hdfs dfs -ls /user/hive/warehouse/painel_covidbr.db/sintese_casos_estados | head -n 5

Found 29 items
-rw-r--r--   2 root supergroup          0 2022-08-08 04:44 /user/hive/warehouse/painel_covidbr.db/sintese_casos_estados/_SUCCESS
drwxr-xr-x   - root supergroup          0 2022-08-08 04:44 /user/hive/warehouse/painel_covidbr.db/sintese_casos_estados/estado=AC
drwxr-xr-x   - root supergroup          0 2022-08-08 04:44 /user/hive/warehouse/painel_covidbr.db/sintese_casos_estados/estado=AL
drwxr-xr-x   - root supergroup          0 2022-08-08 04:44 /user/hive/warehouse/painel_covidbr.db/sintese_casos_estados/estado=AM


# Painel da COVID-19

In [7]:
from pyspark.sql.functions import min, max, col, asc, desc, lit, sum, format_number

In [8]:
df_regioes_estados = spark.read.table('painel_covidbr.sintese_casos_estados')

## Visualização do painel da COVID-19 no Brasil

In [9]:
df_brasil = df_regioes_estados.filter(col('regiao') == 'Brasil')

## (Visão 1) - Casos recuperados e em acompanhamento

In [10]:
df_visao1 = (
    df_brasil
    .select(
        col('recuperadosNovos'),
        col('emAcompanhamentoNovos')
    )
)

In [11]:
df_visao1.show()

+----------------+---------------------+
|recuperadosNovos|emAcompanhamentoNovos|
+----------------+---------------------+
|        17262646|              1065477|
+----------------+---------------------+



In [ ]:
(df_visao1.write
 .mode('overwrite')
 .saveAsTable('painel_covidbr.painel_casos_recuperados_acompanhamento'))

In [13]:
!hdfs dfs -ls /user/hive/warehouse/painel_covidbr.db/painel_casos_recuperados_acompanhamento

Found 2 items
-rw-r--r--   2 root supergroup          0 2022-08-08 02:48 /user/hive/warehouse/painel_covidbr.db/painel_casos_recuperados_acompanhamento/_SUCCESS
-rw-r--r--   2 root supergroup        688 2022-08-08 02:48 /user/hive/warehouse/painel_covidbr.db/painel_casos_recuperados_acompanhamento/part-00000-fa4e788f-3c93-4cbc-82c9-b3da897e8991-c000.snappy.parquet


## (Visão 2) - Casos confirmados

In [16]:
df_visao2 = (
    df_brasil
    .select(
        col('casosAcumulado'), 
        col('casosNovos'), 
        col('incidencia')
    )
)

In [17]:
df_visao2.show()

+--------------+----------+----------------+
|casosAcumulado|casosNovos|      incidencia|
+--------------+----------+----------------+
|      18855015|     62504|8972.29262600000|
+--------------+----------+----------------+



In [ ]:
(df_visao2.write
 .mode('overwrite')
 .option('compression', 'snappy')
 .parquet('/user/hive/warehouse/casos_confirmados_parquet'))

In [18]:
!hdfs dfs -ls /user/hive/warehouse/casos_confirmados_parquet

Found 2 items
-rw-r--r--   2 root supergroup          0 2022-08-08 02:48 /user/hive/warehouse/casos_confirmados_parquet/_SUCCESS
-rw-r--r--   2 root supergroup       1010 2022-08-08 02:48 /user/hive/warehouse/casos_confirmados_parquet/part-00000-9d53944b-3b76-4529-88dd-a695d439cd89-c000.snappy.parquet


## (Visão 3) - Óbitos confirmados

In [19]:
from pyspark.sql.functions import to_json, struct

df_visao3 = (
    df_brasil
    .select(to_json(struct(
        col('obitosAcumulado'),
        col('obitosNovos'),
        col('letalidade'),
        col('mortalidade'))).alias("value")
    )
)

In [20]:
df_visao3.show(1, False)

+---------------------------------------------------------------------------------------------------------+
|value                                                                                                    |
+---------------------------------------------------------------------------------------------------------+
|{"obitosAcumulado":526892,"obitosNovos":1780,"letalidade":0.02794439570,"mortalidade":250.72529543290204}|
+---------------------------------------------------------------------------------------------------------+



In [ ]:
(df_visao3.write
 .format('kafka')
 .option('kafka.bootstrap.servers', 'kafka:29092')
 .option('topic', 'covid19-obitos')
 .save())

Para testar o resultado do kafka, execute:

```bash
sh scripts/teste-kafkaconsumer.sh
```

## Visualização da síntese de casos de COVID-19 no Brasil e Regiões

## (Visão 1) - Síntese de casos do Brasil e Regiões

In [21]:
df_sintese_regioes = (
    df_regioes_estados
    .groupBy('regiao')
    .agg(
        sum('casosAcumulado').alias('casosAcumulado'),
        sum('obitosAcumulado').alias('obitosAcumulado'),
        sum('populacao').alias('populacao'),
        max('data').alias('data')
    )
    .withColumn('incidencia', (col('casosAcumulado') / col('populacao')) * lit(100_000))
    .withColumn('mortalidade', (col('obitosAcumulado') / col('populacao')) * lit(100_000))
    .select(
        col('regiao'),
        col('casosAcumulado'),
        col('obitosAcumulado'),
        col('incidencia'),
        col('mortalidade'),
        col('data')
    )
    .orderBy(desc('casosAcumulado'))
)

In [23]:
df_sintese_regioes.show()

+------------+--------------+---------------+-----------------+------------------+-------------------+
|      regiao|casosAcumulado|obitosAcumulado|       incidencia|       mortalidade|               data|
+------------+--------------+---------------+-----------------+------------------+-------------------+
|      Brasil|      18855015|         526892| 8972.29262594004|250.72529543290204|2021-07-06 00:00:00|
|     Sudeste|       7138803|         245311| 8078.17951758234| 277.5908363961915|2021-07-06 00:00:00|
|    Nordeste|       4455737|         107824| 7807.26803537182|188.92741394878797|2021-07-06 00:00:00|
|         Sul|       3611041|          80705|12046.44691563753| 269.2321960139824|2021-07-06 00:00:00|
|Centro-Oeste|       1916619|          49207|11760.50989275744| 301.9376361670813|2021-07-06 00:00:00|
|       Norte|       1732815|          43845| 9401.64332010560|237.88751330640042|2021-07-06 00:00:00|
+------------+--------------+---------------+-----------------+----------